### Import data

In [1]:
import numpy as np
import pandas as pd
import random
import ast
from collections import defaultdict

In [2]:


PP_recipes = pd.read_csv('Data/PP_recipes.csv')
PP_users = pd.read_csv('Data/PP_users.csv')
Data = []
techPerUser = defaultdict(list)
techPerRecipe = defaultdict(list)


for index, row in PP_users.iterrows():
    for i in ast.literal_eval(row['items']):
        if i in PP_recipes['i'].values:
            Data.append([int(row['u']), int(i), 1])
            techPerUser[int(row['u'])].append(PP_recipes[PP_recipes['i'] == i]['techniques'].values[0])
            if int(i) not in techPerRecipe:
                techPerRecipe[int(i)] = PP_recipes[PP_recipes['i'] == i]['techniques'].values[0]
            

In [3]:
random.shuffle(Data)
length = len(Data)
TrainData = Data[:int(length*0.75)]
ValidData = Data[int(length*0.75):]

In [4]:
       
recipiesPerUser = defaultdict(list)
UsersPerRecipe = defaultdict(list)
for u, i, r in TrainData:
    if int(u) not in recipiesPerUser:
        recipiesPerUser[int(u)] = [int(i)]
    else:
        if int(i) not in recipiesPerUser[int(u)]:
            recipiesPerUser[int(u)].append(int(i))

    if i in UsersPerRecipe:
        if int(u) not in UsersPerRecipe[int(i)]:
            UsersPerRecipe[int(i)].append(u)
    else:
        UsersPerRecipe[int(i)] = [int(u)]
       

In [5]:
newTechPerRecipe = {}
for i in techPerRecipe:
    t =0
    l = ast.literal_eval(techPerRecipe[i])
    
    for j in l:
        if j ==1:
            if i not in newTechPerRecipe:
                newTechPerRecipe[i] = [t]
            else:
                newTechPerRecipe[i].append(t)
        t += 1
    if i not in newTechPerRecipe:
        newTechPerRecipe[i] = [99]

newTechPerUser = {}

for i in techPerUser:
    for j in techPerUser[i]:
        t = 0
        a =0
        for k in ast.literal_eval(j):
            if k ==1:
                a=1
                if i not in newTechPerUser:
                    newTechPerUser[i] = [t]
                else:
                    if t not in newTechPerUser[i]:
                        newTechPerUser[i].append(t)
            t += 1
        if a == 0:
            if i not in newTechPerUser:
                newTechPerUser[i] = [99]
            else:
                if 99 not in newTechPerUser[i]:
                    newTechPerUser[i].append(99)

In [6]:
a = list(UsersPerRecipe.keys())
def ranRecipe(user):
    '''random game user hasnt played'''
    ran = random.choice(a)
    
    # Check if the user hasn't played any games
    if user not in recipiesPerUser:
        return ran

    # Check if the user has played all the games
    if len(recipiesPerUser[user]) == len(a):
        print('all games played')
        return ran
    
    # Keep selecting a random game until it is one that the user hasn't played
    it = 0
    while ran in recipiesPerUser[user]:
        it += 1
        ran = random.choice(a)
        if it > 1000:
            print('infinite loop')
            return ran
    return ran

newTrainData = []
for u,r,y in TrainData:
    newTrainData.append([u,r,y])
    newTrainData.append([u,ranRecipe(u),0])

newValidData = []
for u,r,y in ValidData:
    newValidData.append([u,r,y])
    newValidData.append([u,ranRecipe(u),0])

TrainData = newTrainData
ValidData = newValidData
random.shuffle(TrainData)
random.shuffle(ValidData)


In [7]:
# Initialize a defaultdict to count the number of times each Recipe is played
RecipeCount = {}
# Initialize a variable to keep track of the total number of Recipes played
totalPlayed = 0

# Iterate over the training hours
for user, Recipe, y in TrainData:
  if y == 1:
    # Increment the count for the current Recipe
    if Recipe not in RecipeCount:
        RecipeCount[Recipe] = 0
    RecipeCount[Recipe] += 1
    # Increment the total number of Recipes played
    totalPlayed += 1

# Create a list of tuples containing the Recipe count and Recipe ID
mostPopular = [(RecipeCount[x], x) for x in RecipeCount]
# Sort the list in descending order based on the Recipe count
mostPopular.sort(reverse=True)

# Create a set to store the most popular Recipes
return1 = set()
# Initialize a variable to keep track of the cumulative Recipe count
count = 0

# Iterate over the sorted list of most popular Recipes
for ic, i in mostPopular:
  # Add the Recipe ID to the set of most popular Recipes
  return1.add(i)
  # Increment the cumulative Recipe count
  count += ic
  # Check if the cumulative Recipe count exceeds the threshold (68.6868% of total Recipes played)
  if count > totalPlayed * 0.686868686868687:
    break


In [8]:
y_actual = [d for u,r,d in ValidData]
y_pred = [1 if r in return1 else 0 for u,r,d in ValidData]

truePos = 0
falsePos = 0
trueNeg = 0
falseNeg = 0
for i in range(len(y_pred)):
    if y_pred[i] == 1:
        if y_actual[i] == 1:
            truePos += 1
        else:
            falsePos += 1
    else:
        if y_actual[i] == 1:
            falseNeg += 1
        else:
            trueNeg += 1
print('True Positives: ', truePos)
print('False Positives: ', falsePos)
print('True Negatives: ', trueNeg)
print('False Negatives: ', falseNeg)
print('Accuracy: ', (truePos + trueNeg) / len(y_pred))

True Positives:  107824
False Positives:  42356
True Negatives:  132370
False Negatives:  66902
Accuracy:  0.6873447569337133


In [9]:

# Function to calculate Jaccard similarity between two sets
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))  # Calculate the intersection of the two sets
    denom = len(s1.union(s2))  # Calculate the union of the two sets
    if denom == 0:
        return 0
    return numer / denom  # Return the Jaccard similarity coefficient

# Function to calculate Jaccard similarity between two sets
def Jaccard1(s1, s2):
    numer = len(s1.intersection(s2))  # Calculate the intersection of the two sets
    denom = len(s2)  # Calculate the union of the two sets
    if denom == 0:
        return 0
    return numer / denom  # Return the Jaccard similarity coefficient

def trainJaccard(thres_start=1, thres_end=57, N_Tstep=500):
    sim1 = []
    b=0
    
    for u,r,_ in TrainData:
        
        if u not in newTechPerUser or r not in newTechPerRecipe:
            sim1.append(0)
            b+=1
        else:
            sim1.append(Jaccard1(set(newTechPerUser[u]), set(newTechPerRecipe[r])))
    print(b)
    best_threshold = 0
    best_acc = 0
    y_actual = [d for _, _, d in TrainData]
    for thres in np.linspace(thres_start, thres_end, N_Tstep):
        y_pred = (np.array(sim1) < thres) & (np.array(sim1) > 1)
        acc = sum(y_pred == y_actual) / len(y_actual)
        if acc > best_acc:
            best_acc = acc
            best_threshold = thres

    print('best threshold: ', best_threshold, 'Acc: ', best_acc)

    return best_threshold

def predictTech(u, r, thres=0.2038):
    if u not in newTechPerUser or r not in newTechPerRecipe:
        return 0
    sim = Jaccard1(set(newTechPerUser[u]),set(newTechPerRecipe[r]))
    if sim >= 1:
        return 1
    return 0


In [14]:
y_actual = [d for _, _, d in ValidData]
y_pred = []
for u, r, _ in ValidData:
    y_pred.append(predictTech(int(u), int(r)))

truePos = 0
falsePos = 0
trueNeg = 0
falseNeg = 0
for i in range(len(y_pred)):
    if y_pred[i] == 1:
        if y_actual[i] == 1:
            truePos += 1
        else:
            falsePos += 1
    else:
        if y_actual[i] == 1:
            falseNeg += 1
        else:
            trueNeg += 1
print('True Positives: ', truePos)
print('False Positives: ', falsePos)
print('True Negatives: ', trueNeg)
print('False Negatives: ', falseNeg)
print('Accuracy: ', (truePos + trueNeg) / len(y_pred))
BER = 1 - 0.5 * ((truePos / (truePos + falseNeg)) + (trueNeg / (trueNeg + falsePos)))
print('BER: ', BER)

True Positives:  174726
False Positives:  142266
True Negatives:  32460
False Negatives:  0
Accuracy:  0.5928882936712339
BER:  0.40711170632876614


In [15]:
y_actual = [d for u,r,d in ValidData]
y_pred = [1*predictTech(int(u),int(r)) if r in return1 else 0 for u,r,d in ValidData]
truePos = 0
falsePos = 0
trueNeg = 0
falseNeg = 0
for i in range(len(y_pred)):
    if y_pred[i] == 1:
        if y_actual[i] == 1:
            truePos += 1
        else:
            falsePos += 1
    else:
        if y_actual[i] == 1:
            falseNeg += 1
        else:
            trueNeg += 1
print('True Positives: ', truePos)
print('False Positives: ', falsePos)
print('True Negatives: ', trueNeg)
print('False Negatives: ', falseNeg)
print('Accuracy: ', (truePos + trueNeg) / len(y_pred))
BER = 1 - 0.5 * ((truePos / (truePos + falseNeg)) + (trueNeg / (trueNeg + falsePos)))
print('BER: ', BER)

True Positives:  107824
False Positives:  34522
True Negatives:  140204
False Negatives:  66902
Accuracy:  0.7097627141925071
BER:  0.29023728580749286


In [12]:
# Function to calculate Jaccard similarity between two sets
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))  # Calculate the intersection of the two sets
    denom = len(s1.union(s2))  # Calculate the union of the two sets
    if denom == 0:
        return 0
    return numer / denom  # Return the Jaccard similarity coefficient

# Function to calculate the similarity between a user and a game
def similar(u, g):
    siml = []
    if g not in UsersPerRecipe:
        return random.randint(0, 1)
    
    if predictTech(u, g) ==0:
        return 0
    
    for i in recipiesPerUser[u]:
        if i == g:
            continue
        
        siml.append(Jaccard(set(UsersPerRecipe[g]), set(UsersPerRecipe[i])))

    if len(siml) == 0:
        return 0

    siml.sort(reverse=True)

    if len(siml) // 2 > 1:
        siml = siml[:len(siml) // 2]
    else:
        return siml[0]

    simAvg = sum(siml) / len(siml)
    return simAvg

# Function to train the model by finding the best threshold value
def trainModel(thres_start=0.005, thres_end=0., N_Tstep=100):
    sim1 = getSim(TrainData)
    print('sim1 done')
    return sim1 

def trainThreshold(sim1,thres_start=0.005, thres_end=0.015, N_Tstep=100, s_start=1, s_end=1.1, N_Sstep=3):
    best_threshold = 0
    best_acc = 0
    y_actual = [d for _, _, d in TrainData]
    for thres in np.linspace(thres_start, thres_end, N_Tstep):
        y_pred = np.array(sim1) > thres
        acc = sum(y_pred == y_actual) / len(y_actual)
        if acc > best_acc:
            best_acc = acc
            best_threshold = thres
    print('best threshold: ', best_threshold, 'Acc: ', best_acc)
    return best_threshold

# Function to validate the model using a given threshold value
def testModel(thres,Data=TrainData):
    sim1 = getSim(Data)
    y_actual = [d for _, _, d in Data]
    y_pred = np.array(sim1) > thres
    acc = sum(y_pred == y_actual) / len(y_actual)
    print('Accuracy: ', acc)
    return acc

# Function to calculate the similarity between a user and all games in the validation set
def getSim(Data):
    sim3 = []
    it = 0
    for u, g, _ in Data:
        it += 1
        if it % 10000 == 0:
            print(it)
        if u not in recipiesPerUser:
            sim3.append(0)
        else:
            sim3.append(similar(u, g))
    return sim3

# Function to predict whether a user will play a game based on similarity and threshold values
def predict(u, g, thres=0.00931):
    if u not in recipiesPerUser:
        return random.randint(0, 1)
    sim = similar(u, g)
    if sim > thres:
        return 1
    return 0

In [13]:
#testModel(0.00931,TrainData)
testModel(0.1,ValidData)


10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
Accuracy:  0.5312947128647139


0.5312947128647139